### 안녕하세요, 한국투자증권 신입사원 DT프로젝트
### AI 모델링 경진대회에 참여하신 여러분을 환영합니다.

### 베이스라인은 모든 참가자의 ***'제출'***을 목표로 제공해 드립니다.
### 베이스라인을 활용하여 여러분만의 고도화된 모델을 만들어 보세요.



---

## 0. 환경 준비
- ### 라이브러리 호출

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

---

## 1. 데이터 불러오기
- ### train.csv, test.csv 불러오기

In [ ]:
# 파일 경로는 여러분의 데이터 저장소로 변경
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")

- ### 데이터 탐색하기

In [ ]:
# 데이터 샘플 확인하기
train.head(3)

In [ ]:
# 데이터 정보 확인하기
train.info()

In [ ]:
# 수치형 데이터 분석하기
train.describe()

## 2. 데이터 전처리하기
- ### 결측치 처리하기

In [ ]:
# 결측치 확인하기
train.isnull().sum()

In [ ]:
# 결측치 이전값('ffill') 로 채우기
train.fillna(method='ffill', inplace=True)

- ### Label-Encoding(Object형 데이터 수치형으로 변환하기)

In [ ]:
# object 컬럼 리스트로 저장하기 
object_columns = ['workclass', 'education_level', 'marital-status', 'occupation', 
                  'relationship', 'race', 'sex', 'native-country']

# Label-Encoding하기
le = LabelEncoder()
le_list = []

for c in object_columns:    
    le = le.fit(test[c])
    train[c] = le.transform(train[c])   #train['col']에 따라 encoding
    test[c] = le.transform(test[c])   #train['col']에 따라 encoding
    le_list.append(le.classes_)

In [ ]:
# Label Encoding 된 Column 매핑 값 확인하기
le_list = pd.DataFrame(le_list).transpose()
le_list.columns = object_columns
le_list

## 3. 데이터 분리하기
- ### feature(X)와 target(y) 분리하기
모델을 학습하기 위해서는 feature와 target로 나누어야 합니다.<br>
일반적으로 모델에는 정수, 실수형태의 값만 사용할 수 있습니다.<br>
해당 데이터에서는 target(y)는 ***'income'*** 이고,<br>
feature(X)는 ***'income'*** 을 제외한 값입니다.<br>

In [ ]:
# X는 독립변수이므로 종속변수를 제거합니다. 또한 target 이외의 문자열 데이터를 제거합니다.
X = train.drop(["income"], axis = 1)
# y는 종속변수로 값을 설정합니다.
y = train['income']

- ### 학습데이터(train) 와 검증데이터(valid) 분리하기
모델의 성능을 중간적으로 점검하기 위해 검증데이터(validation data)를 생성하겠습니다.

In [ ]:
# 학습데이터, 검증데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2)

---

## 4. 모델링하기
- ### 모델 선언 및 학습하기
RandomForest는 tree기반 모델로, DecisionTree를 여러 개 생성하여 그들의 평균으로 예측의 성능을 높이는 Ensemble Machine Learning 모델입니다.<br>
RandomForest에 대해 더 자세히 알고싶으시다면 아래의 링크를 참고해주시기 바랍니다.

In [ ]:
# 모델 선언
model = RandomForestClassifier(random_state = 42)

# 모델 학습
model.fit(X_train,y_train)

- ### 모델 검증하기
모델이 어느정도 정답을 맞추는지 검증을 통해 대략적으로 확인할 수 있습니다.<br>
평가 지표인 Accuracy로 검증을 수행하겠습니다.<br>
모델에 검증데이터를 입력하여 검증 점수를 예측합니다.

In [ ]:
val_pred = model.predict(X_valid)

# 모델 성능 검증
print("Model Accuratcy : {}".format(accuracy_score(val_pred, y_valid)))

- ### test 데이터 예측하기

In [ ]:
# 최종 예측을 하기위해 test값을 입력합니다.
test_pred = model.predict(test)

---

## 5. 제출하기
- ### 양식에 맞춰 Submission 파일 만들기

In [ ]:
# 제출 파일 양식을 불러옵니다.
submission = pd.read_csv('./example_submission.csv')

# 'income' 컬럼에 예측 결과('test_pred')를 입력하기 
submission['income'] = test_pred

- ### 최종 제출 파일 만들기

In [ ]:
# 최종 submission 파일 저장하기
submission.to_csv('./my_submission.csv', index=False)